In [1]:
import pickle
import pandas as pd
import numpy as np
import utilities
import preprocess
import parameters

from sklearn.metrics import accuracy_score, hamming_loss
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
from sentence_transformers import util, SentenceTransformer
from sklearn.metrics import hamming_loss, accuracy_score, f1_score, classification_report
from sklearn.utils.class_weight import compute_class_weight
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression

In [2]:
import warnings
warnings.filterwarnings("ignore")

## parameters

In [3]:
# importing algorithm parameters
sim_type = parameters.sim_type
random_state = 2
test_size = parameters.test_size

# doe
balance_ratio = parameters.balance_ratio
sim_calculation_type = parameters.sim_calculation_type

success_metric = parameters.success_metric
embedding_method = parameters.embedding_method
data_paths = parameters.data_paths
X_num_paths = parameters.X_num_paths
unlabeled_ratios = parameters.unlabeled_ratios

np.random.seed(random_state)

In [4]:
from sklearn.svm import SVC

In [5]:
classifier_object = SVC(kernel='linear',probability=True, class_weight='balanced')
# classifier_object = LogisticRegression(class_weight='balanced')

In [6]:
def split_data(X, y):
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_state)
    X_labeled, X_unlabeled, y_labeled, y_unlabeled = train_test_split(X_train, y_train, test_size=unlabeled_ratios[data], 
                                                                  random_state=random_state)
    
    return X_labeled, y_labeled, X_unlabeled, y_unlabeled, X_test, y_test

In [7]:
def split_data_KFold(X, y, cv):
    
    from sklearn.model_selection import KFold, StratifiedKFold
    
    sorted_y = y.sum().sort_values(ascending=False)
    for col, _ in zip(sorted_y.index, sorted_y):
        y.loc[y[y[col]==1].index, 'dominant_label'] = col
        
    stratify_flag = y['dominant_label']
    y.drop(['dominant_label'], axis=1, inplace=True)
    
    kf = StratifiedKFold(n_splits=cv, random_state=random_state, shuffle=True)
    
    splits = []
    for train_idx, test_idx in kf.split(X.index, stratify_flag):
        
        labeled_idx, unlabeled_idx = train_test_split(train_idx, test_size=unlabeled_ratios[data], random_state=random_state)
        
        splits.append((labeled_idx, unlabeled_idx, test_idx))
        
    return splits    

In [8]:
def read_data(data):
    # reading data
    df = utilities.read_data(data_paths[data])
    # X = df['text'].apply(preprocess.preprocess_text)
    y = df.drop(['text'], axis=1)
    # ------------------------------------------------------------------------------------------------------------------------------
    # reading from a pickle instead of applying vectorization
    # X_num = utilities.vectorize_data(X, embedding_method)
    # X_num = pd.Series([np.squeeze(i) for i in X_num])
    X = pd.read_pickle(X_num_paths[data])
    
    return X, y

In [9]:
def reduce_dimension(X, n_comp=200):
    
    X_ = np.vstack(X.values)

    from sklearn.decomposition import TruncatedSVD
    svd = TruncatedSVD(n_components=n_comp)
    X_reduced = svd.fit_transform(X_)
    
    return pd.Series([np.squeeze(i) for i in X_reduced], index=X.index)

# main

In [10]:
def main(data, balance_ratio, sim_calculation_type, single_metric, oversampler_version, batch_size, n_iter, sim_type):
    
    print('*'*100)
    print('\x1b[1;31m'+data+'\x1b[0m')
    
    X, y = read_data(data)
    random_state=6
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_state)
    X_labeled, X_unlabeled, y_labeled, y_unlabeled = train_test_split(X_train, y_train, test_size=unlabeled_ratios[data], 
                                                                  random_state=random_state)
    print(y_labeled.sum())
    shape_before = X_labeled.shape[0]
    print(X_labeled.shape, X_unlabeled.shape, X_test.shape)
    s_metric = utilities.multilabel_classifier(np.vstack(X_labeled), y_labeled, np.vstack(X_test), y_test, 
                                               success_metric=success_metric,
                                               classifier_object = classifier_object, 
                                               print_results=True)
    # -----------------------------------------------------------------------------------------------------------------------------
    # calculation number of instances to balance dataset
    num_of_new_instances = utilities.calculate_balancing_num_instance_multiclass(y_labeled, balance_ratio, 
                                                                                 calculation_type='metric_based', 
                                                                                 s_metrics=s_metric)
    # -----------------------------------------------------------------------------------------------------------------------------
    # oversampling dataset using unlabeled data with the given ratios
    # print('num_of_new_instances : ',num_of_new_instances)
    if oversampler_version == 'v1':
        validation, X_labeled, y_labeled, X_unlabeled, y_unlabeled = utilities.oversample_dataset_v1(
                                                                        num_of_new_instances, X_labeled, y_labeled, 
                                                                        X_unlabeled, y_unlabeled, X_test, y_test, 
                                                                        sim_calculation_type=sim_calculation_type,
                                                                        batch_size=batch_size)
    elif oversampler_version == 'v2':
        validation, X_labeled, y_labeled, X_unlabeled, y_unlabeled = utilities.oversample_dataset_v2(
                                                                        num_of_new_instances, X_labeled, y_labeled, 
                                                                        X_unlabeled, y_unlabeled, X_test, y_test, 
                                                                        sim_calculation_type=sim_calculation_type,
                                                                        batch_size=batch_size)
    elif oversampler_version == 'v3':
        validation, X_labeled, y_labeled, X_unlabeled, y_unlabeled = utilities.oversample_dataset_v3(
                                                                        num_of_new_instances, X_labeled, y_labeled, 
                                                                        X_unlabeled, y_unlabeled, X_test, y_test, 
                                                                        sim_calculation_type=sim_calculation_type,
                                                                        batch_size=batch_size, 
                                                                        n_iter=n_iter, 
                                                                        single_score=single_metric)
    elif oversampler_version == 'v4':
        validation, X_labeled, y_labeled, X_unlabeled, y_unlabeled, metric_history = utilities.oversample_dataset_v4(
                                                                                         num_of_new_instances, 
                                                                                         X_labeled, y_labeled, 
                                                                                         X_unlabeled, y_unlabeled, 
                                                                                         X_test, y_test, 
                                                                                         sim_calculation_type=sim_calculation_type, 
                                                                                         batch_size=batch_size, 
                                                                                         n_iter=n_iter,
                                                                                         balance_ratio=balance_ratio,
                                                                                         success_metric=success_metric,
                                                                                         single_score=single_metric,
                                                                                         sim_type=sim_type)
    # -----------------------------------------------------------------------------------------------------------------------------
    # check if the result gets better
    shape_after = X_labeled.shape[0]
    s_metric = utilities.multilabel_classifier(np.vstack(X_labeled), y_labeled, np.vstack(X_test), y_test, 
                                               success_metric=success_metric,
                                               classifier_object = classifier_object, 
                                               print_results=True)
    # comparing the found labels and ground truth
    y_true, y_pred = [], []
    for _, _, _, y_t, y_p in validation:
        y_true.append(list(y_t.values))
        y_pred.append(list(y_p.values()))
    
    acc = 1-hamming_loss(y_true, y_pred)
    emr = accuracy_score(y_true, y_pred)  
    print('-'*30)
    print(f'Shape: before {shape_before}, after {shape_after} : {shape_after-shape_before} instances added...')
    print(f'Exact match ratio : {emr:.2f} ')
    print(f'Accuracy          : {acc:.2f} ')
    print(classification_report(y_true, y_pred))
    print('-'*30)
    
    if oversampler_version == 'v4':
        return metric_history

In [11]:
def run_CV(data, balance_ratio, sim_calculation_type, single_metric, oversampler_version, batch_size, n_iter, sim_type):
    
    CV_results = []
    
    X, y = read_data(data)
    X = reduce_dimension(X)        

    splits = split_data_KFold(X, y, 5)

    for labeled_idx, unlabeled_idx, test_idx in splits:

        X_labeled = X.loc[labeled_idx]
        y_labeled = y.loc[labeled_idx]
        X_unlabeled = X.loc[unlabeled_idx]
        y_unlabeled = y.loc[unlabeled_idx]
        X_test = X.loc[test_idx]
        y_test = y.loc[test_idx]
    
    
        shape_before = X_labeled.shape[0]

        s_metric_before, initial_scores = utilities.multilabel_classifier(np.vstack(X_labeled), y_labeled, np.vstack(X_test), y_test, 
                                                   success_metric=success_metric,
                                                   classifier_object = classifier_object, 
                                                   print_results=False, return_scores=True)
        # -----------------------------------------------------------------------------------------------------------------------------
        # calculation number of instances to balance dataset
        num_of_new_instances = utilities.calculate_balancing_num_instance_multiclass(y_labeled, balance_ratio, 
                                                                                     calculation_type='metric_based', 
                                                                                     s_metrics=s_metric_before)
        # -----------------------------------------------------------------------------------------------------------------------------
        # oversampling dataset using unlabeled data with the given ratios
        # print('num_of_new_instances : ',num_of_new_instances)
        if oversampler_version == 'v1':
            validation, X_labeled, y_labeled, X_unlabeled, y_unlabeled = utilities.oversample_dataset_v1(
                                                                            num_of_new_instances, X_labeled, y_labeled, 
                                                                            X_unlabeled, y_unlabeled, X_test, y_test, 
                                                                            sim_calculation_type=sim_calculation_type,
                                                                            batch_size=batch_size)
        elif oversampler_version == 'v2':
            validation, X_labeled, y_labeled, X_unlabeled, y_unlabeled = utilities.oversample_dataset_v2(
                                                                            num_of_new_instances, X_labeled, y_labeled, 
                                                                            X_unlabeled, y_unlabeled, X_test, y_test, 
                                                                            sim_calculation_type=sim_calculation_type,
                                                                            batch_size=batch_size)
        elif oversampler_version == 'v3':
            validation, X_labeled, y_labeled, X_unlabeled, y_unlabeled = utilities.oversample_dataset_v3(
                                                                            num_of_new_instances, X_labeled, y_labeled, 
                                                                            X_unlabeled, y_unlabeled, X_test, y_test, 
                                                                            sim_calculation_type=sim_calculation_type,
                                                                            batch_size=batch_size, 
                                                                            n_iter=n_iter, 
                                                                            single_score=single_metric)
        elif oversampler_version == 'v4':
            validation, X_labeled, y_labeled, X_unlabeled, y_unlabeled, metric_history = utilities.oversample_dataset_v4(
                                                                                             num_of_new_instances, 
                                                                                             X_labeled, y_labeled, 
                                                                                             X_unlabeled, y_unlabeled, 
                                                                                             X_test, y_test, 
                                                                                             sim_calculation_type=sim_calculation_type, 
                                                                                             batch_size=batch_size, 
                                                                                             n_iter=n_iter,
                                                                                             balance_ratio=balance_ratio,
                                                                                             success_metric=success_metric,
                                                                                             single_score=single_metric,
                                                                                             sim_type=sim_type)
        # -----------------------------------------------------------------------------------------------------------------------------
        # check if the result gets better
        shape_after = X_labeled.shape[0]
        s_metric_after, final_scores = utilities.multilabel_classifier(np.vstack(X_labeled), y_labeled, np.vstack(X_test), y_test, 
                                                   success_metric=success_metric,
                                                   classifier_object = classifier_object, 
                                                   print_results=False, return_scores=True)
        # comparing the found labels and ground truth
        y_true, y_pred = [], []
        for _, _, _, y_t, y_p in validation:
            y_true.append(list(y_t.values))
            y_pred.append(list(y_p.values()))
        
        acc = 1-hamming_loss(y_true, y_pred)
        emr = accuracy_score(y_true, y_pred)  
        '''
        print('-'*30)
        print(f'Shape: before {shape_before}, after {shape_after} : {shape_after-shape_before} instances added...')
        print(f'Exact match ratio : {emr:.2f} ')
        print(f'Accuracy          : {acc:.2f} ')
        print(classification_report(y_true, y_pred))
        print('-'*30)
        '''
        if y_true and y_pred:
            clf_report = classification_report(y_true, y_pred)
        else:
            clf_report = ''
            
        CV_results.append({'shape_before':shape_before,'shape_after':shape_after, 'val_accuracy':acc, 'val_exact_match':emr, 
                          's_metric_before':s_metric_before, 's_metric_after':s_metric_after, 'initial_scores':initial_scores,
                          'final_scores':final_scores, 'validation':validation, 'clf_report':clf_report})
        
    return CV_results

In [12]:
def main_CV(data, balance_ratio, sim_calculation_type, single_metric, oversampler_version, batch_size, n_iter, sim_type):
    
    CV_results = run_CV(data, balance_ratio, sim_calculation_type, single_metric, oversampler_version, batch_size, n_iter, sim_type)
    
    return CV_results

data = 'opp115'
balance_ratio = 0.5
sim_calculation_type = 'average'
single_metric = 'f1_score'
batch_size = 1
n_iter = 200
sim_type = 'cosine'

CV_res = main_CV(data, balance_ratio, sim_calculation_type, single_metric, 'v4', batch_size, n_iter, sim_type)

for i in CV_res:
    print(i['initial_scores']['f1_score']) 
    print(i['final_scores']['f1_score'])
    print(i['shape_before'], i['shape_after'])
    print('-----')

### experimentations

In [13]:
parameters = {}
parameters['data'] = ['opp115', 'ohsumed', 'reuters']
parameters['balance_ratio'] = [0.2, 0.5]
parameters['sim_calculation_type'] = ['average', 'safe_interval']
parameters['batch_size'] = [1, 3, 5]
parameters['n_iter'] = [100, 500]
parameters['sim_type'] = ['cosine', 'euclidean', 'JS']

In [14]:
run_size = 10
data = 'reuters'
single_metric = 'f1_score'
file_name = 'results_reuters_1.p'

In [15]:
all_param_grid = []
for balance_ratio in parameters['balance_ratio']:
    for sim_calculation_type in parameters['sim_calculation_type']:
        for batch_size in parameters['batch_size']:
            for n_iter in parameters['n_iter']:
                for sim_type in parameters['sim_type']:
                    for run_num in range(1,run_size+1):
                        param_list = [run_num, balance_ratio, sim_calculation_type, batch_size, n_iter, sim_type, single_metric]
                        all_param_grid.append(param_list)

results_df = pd.DataFrame(all_param_grid, columns=['run', 'balance_ratio', 'sim_calculation_type', 'batch_size', 'n_iter', 'sim_type','single_metric'])
results_df['result'] = '-'
results_df.to_pickle(file_name)

In [17]:
results_df = pd.read_pickle(file_name)
iter_df = results_df[results_df['result']=='-']
iter_df = iter_df[iter_df['sim_type']=='cosine']

In [18]:
iter_df.shape

(240, 8)

In [ ]:
for idx, row in iter_df.iterrows():
    
    run_num, balance_ratio, sim_calculation_type, batch_size, n_iter, sim_type, single_metric, _ = row
    CV_res = main_CV(data, balance_ratio, sim_calculation_type, single_metric, 'v4', batch_size, n_iter, sim_type)
    
    results_df.loc[idx, 'result'] = str(CV_res)
    
    results_df.to_pickle(file_name)